In [1]:
from dask_jobqueue import SLURMCluster

# Compose SLURM script
cluster = SLURMCluster(queue='broadwl', cores=10, memory='40GB', 
                       processes=10, walltime='01:00:00', interface='ib0',
                       job_extra=['--account=macs30123']
                      )

# Request resources
cluster.scale(jobs=1)

/home/zhiyunhu/.local/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
import pandas as pd
import numpy as np

In [5]:
from dask.distributed import Client

client = Client(cluster)
client

<Client: 'tcp://172.25.220.72:38584' processes=10 threads=10, memory=37.30 GiB>

In [6]:
import dask.dataframe as dd

lyrics_raw = dd.read_csv('lyrics_info_raw.csv')

In [7]:
lyrics_raw.head()

,Year,Artist,Song Name,Lyrics
0,1958,Boots Brown And His Blockbusters,Cerveza,Cerveza LyricsInstrumental\nInstrumentalEmbed
1,1958,Doris Day,Tunnel Of Love,Tunnel Of Love LyricsGetting crazy on the walt...
2,1958,Billy & Lillie,Lucky Ladybug,"Lucky Ladybug LyricsLucky Ladybug, lucky ladyb..."
3,1958,Nappy Brown,It Don't Hurt No More,Damn Lyrics\nOhhhhhhhhh! That's right\nYour gi...
4,1958,Cozy Cole,Topsy I,NAGASAKI LyricsHook:\nIm back in this bitch an...


In [8]:
#drop the na values that occurs
lyrics_raw = lyrics_raw.dropna(subset = ['Lyrics'])

In [9]:
lyrics_raw.head()

,Year,Artist,Song Name,Lyrics
0,1958,Boots Brown And His Blockbusters,Cerveza,Cerveza LyricsInstrumental\nInstrumentalEmbed
1,1958,Doris Day,Tunnel Of Love,Tunnel Of Love LyricsGetting crazy on the walt...
2,1958,Billy & Lillie,Lucky Ladybug,"Lucky Ladybug LyricsLucky Ladybug, lucky ladyb..."
3,1958,Nappy Brown,It Don't Hurt No More,Damn Lyrics\nOhhhhhhhhh! That's right\nYour gi...
4,1958,Cozy Cole,Topsy I,NAGASAKI LyricsHook:\nIm back in this bitch an...


# Remove the Non-English Songs

In [10]:
lyrics_raw[['Lyrics']] = lyrics_raw[['Lyrics']].astype(str)
lyrics_raw = lyrics_raw[lyrics_raw.Lyrics.map(lambda x: x.isascii())]

In [11]:
lyrics_raw.head()

,Year,Artist,Song Name,Lyrics
0,1958,Boots Brown And His Blockbusters,Cerveza,Cerveza LyricsInstrumental\nInstrumentalEmbed
1,1958,Doris Day,Tunnel Of Love,Tunnel Of Love LyricsGetting crazy on the walt...
2,1958,Billy & Lillie,Lucky Ladybug,"Lucky Ladybug LyricsLucky Ladybug, lucky ladyb..."
3,1958,Nappy Brown,It Don't Hurt No More,Damn Lyrics\nOhhhhhhhhh! That's right\nYour gi...
5,1958,The Four Coins,The World Outside,Angel Station LyricsIt's the coldest night of ...


In [12]:
lyrics_raw = lyrics_raw.rename(columns = {'Song Name':'song_name'})

# Remove the songs that the lyrics and song name are not match

In [13]:
#remove the columns where there is no related songs and lyrics
lyrics_raw['check'] = lyrics_raw.apply(lambda x: x.song_name in x.Lyrics and "Lyrics" in x.Lyrics, axis =1, 
                                       meta = pd.Series(name = "check", dtype = np.bool))
#drop all the values if the check value equals to false
lyrics_raw = lyrics_raw.loc[lyrics_raw['check']==True]

/software/Anaconda3-2019.03-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


# Remove the stop words

In [14]:
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS

In [15]:
lyrics_raw['lower_text'] = lyrics_raw['Lyrics'].apply(lambda x: " ".join(x.lower() for x in x.split()),
                                                     meta = ('lower_text', 'object'))
lyrics_raw['text_nonpuc'] = lyrics_raw['lower_text'].str.replace('[^\w\s]', '', regex = True)
lyrics_raw['clean_text'] = lyrics_raw['text_nonpuc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words), 
                                                           meta = ('clean_text', 'object'))

In [16]:
other_stop_words =["im", "dont", "like", 'lyrics','lyric','too', 'hence', 'give', 'never', 'without', "'m", 'something', "'s", 'well', 'hers', 'nine', 'off', 'noone', 'used', 'onto', 'to', 'much', 'us', 'via', 'less', 'had', 'anyone', 'neither', 'an', 'full', 'then', 'twenty', 'thereby', 'keep', 'yourself', 'thus', 'using', 'yourselves', 'each', 'which', 'how', 'doing', '’re', 'whereby', 'with', 'somewhere', 'name', 'that', 'otherwise', 'of', 'around', 'about', 'side', 'sometime', 'back', 'become', 'thru', 'ca', 'becoming', 'formerly', 'empty', 'not', 'often', 'such', 'very', 'therein', 'among', 'here', 'other', 'below', 'again', 'either', 'we', 'what', '’d', 'hereafter', 'ours', 'fifty', 'ourselves', 'your', 'front', 'by', 'upon', 'myself', 'or', 're', 'two', 'thence', 'hundred', 'anywhere', 'make', 'toward', 'towards', 'might', 'thereupon', 'should', 'forty', 'everything', 'me', 'unless', 'nevertheless', 'anyhow', 'former', 'behind', 'always', 'before', 'except', '‘ll', 'wherever', 'nowhere', 'first', 'do', 'move', 'any', 'say', 'been', 'who', 'at', 'also', 'themselves', 'somehow', 'can', 'will', '’s', 'wherein', 'least', 'namely', 'in', 'meanwhile', 'beyond', 'alone', 'moreover', 'and', 'due', 'himself', 'where', 'does', 'she', 'must', 'just', 'hereupon', 'herself', 'n‘t', 'being', 'under', "n't", 'why', 'another', 'during', 'his', 'no', "'ll", '’ve', 'everywhere', 'latterly', 'them', 'seems', 'would', 'next', 'this', 'put', 'third', 'made', 'together', 'as', 'enough', 'but', 'whole', 'you', 'rather', 'our', 'call', 'some', 'between', 'every', 'the', 'everyone', 'their', 'were', 'please', 'six', 'whose', 'perhaps', 'up', 'done', 'whom', 'thereafter', '‘ve', 'indeed', '‘s', 'twelve', '’m', 'sometimes', 'is', 'becomes', 'its', 'may', 'down', 'seemed', 'all', 'many', 'further', 'though', 'through', 'eight', 'are', 'became', 'they', 'yet', 'whither', 'against', 'cannot', 'bottom', 'whereafter', 'none', 'only', 'latter', 'since', 'else', 'was', 'more', 'various', 'serious', 'sixty', "'re", 'take', 'part', 'beside', 'most', 'ever', 'he', 'out', 'over', 'my', 'quite', 'however', "'d", 'because', 'already', 'on', 'four', 'nothing', 'n’t', 'whereas', 'has', 'once', 'top', 'seeming', 'within', 'regarding', 'almost', 'afterwards', 'both', 'per', 'several', 'seem', 'am', 'see', 'whatever', 'did', 'five', 'ten', 'yours', 'really', 'a', 'anything', '‘re', 'mine', 'still', 'elsewhere', 'whence', 'from', 'nor', 'him', 'last', 'her', 'while', 'hereby', 'eleven', 'itself', 'same', 'have', 'across', 'these', 'above', 'so', '‘m', 'therefore', 'than', "'ve", 'others', 'whenever', 'few', 'one', 'into', '‘d', 'get', 'amongst', 'if', 'anyway', 'whether', 'three', 'for', 'fifteen', 'after', 'nobody', 'amount', 'whereupon', 'whoever', 'could', 'it', 'when', 'someone', 'mostly', 'besides', 'although', 'now', 'go', 'until', 'i', 'own', 'herein', 'be', '’ll', 'show', 'throughout', 'even', 'there', 'beforehand', 'those', 'along']

In [17]:
lyrics_raw['further_clean_text'] = lyrics_raw['clean_text'].apply(lambda x: " ".join(x for x in x.split() if x not in other_stop_words),
                                                                     meta = ('further_clean_text', 'object'))

In [18]:
lyrics_clean = lyrics_raw[["Year", "Artist", "song_name", "further_clean_text"]]

In [19]:
lyrics_clean.head()

,Year,Artist,song_name,further_clean_text
0,1958,Boots Brown And His Blockbusters,Cerveza,cerveza lyricsinstrumental instrumentalembed
1,1958,Doris Day,Tunnel Of Love,tunnel love lyricsgetting crazy waltzers life ...
2,1958,Billy & Lillie,Lucky Ladybug,lucky ladybug lyricslucky ladybug lucky ladybu...
6,1958,Frank Sinatra,Mr. Success,mr success lyricswhen walk jam knows head ches...
9,1958,Bobby Helms,The Fool And The Angel,fool angel lyricshere story fool angel fool an...


In [20]:
#filter the top 100 songs for each year
top_100_lyrics = lyrics_clean.compute().groupby('Year').head(100)

# Save the data by each year, decade

In [21]:
#save the clean data to csv
top_100_lyrics.to_csv("top_100_lyrics.csv")

In [22]:
#save each year data
for year in range(1958, 2022):
    temp = top_100_lyrics[top_100_lyrics.Year == year]
    filename = f'{year}.csv'
    temp.to_csv(filename)

In [23]:
#save each decade data
x = range(1958,2018,10)
year_range = []
for i in x:
    start = i
    end = i + 10
    year_range.append([start, end])

In [25]:
for years in year_range:
    filter_decade = (top_100_lyrics['Year'] >= years[0]) & (top_100_lyrics['Year'] < years[1])
    filter_df = top_100_lyrics.loc[filter_decade]
    filename = f'{years[0]}_{years[1]}.csv'
    filter_df.to_csv(filename)

In [27]:
#save the data for the year from 2018 to 2021
filter_last = (top_100_lyrics['Year'] >= 2018) & (top_100_lyrics['Year'] < 2022)
filter_df_last = top_100_lyrics.loc[filter_last]
filter_df_last.to_csv("2018_2021.csv")